# 핵심 키워드 추출 (Keyword Extraction)

# 데이터 준비

## Mecab 설치 (필요시)

In [ ]:
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
0 upgraded, 0 newly installed, 0 to remove and 81 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 81 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.13).
0 upgraded, 0 newly installed, 0 to remove and 81 not upgraded.
mecab-ko i

# TF-IDF 활용 핵심키워드 추출

## 실습 1. sklearn 활용


In [1]:
# 기사 크롤링
import requests 
from bs4 import BeautifulSoup
import re

def get_news_by_url(url):
    headers={"user-agent":"Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, "html.parser")
    content = soup.select_one("#articleBodyContents").get_text().replace("\n", "")
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
  
    start_pos = re.search(r"\w+@\w+\.\w+(.\w+)?", content).start() #뒤에 이메일 형식
    content = content[:start_pos-1]
    return content

    
    
    
docs = []
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

5

## 1) 전처리

In [2]:
from konlpy.tag import Mecab
mecab = Mecab()
preprocessed_docs = []

for doc in docs:
    token_list = []
    for token in mecab.pos(doc):
        if token[1] in ['NNG', 'VV']:
            token_list.append(token[0])
    preprocessed_docs.append(" ".join(token_list))
            
preprocessed_docs[0][:100]

'과기 정통부 장관 참석 기념행사 투입 여종 데이터 구축 민간 외부 연계 체계 개방 강화 기자 국가 차원 빅 데이터 활용 시대 산업 창출 기존 산업 변화 이르 혁신 장 센터 문 분야'

### 2) TF-IDF 계산

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# max_features => vocabulary 개수
count_vect = CountVectorizer(max_df = 0.85, max_features = 10000)
word_count = count_vect.fit_transform(preprocessed_docs) # 훈련(fit) 시키고 적용(transform)
print((count_vect.get_feature_names()[:10]))

['가공', '가능', '가입자', '가족', '가중치', '가치', '각종', '감소', '감염', '강국']


In [4]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count) # 훈련 

TfidfTransformer()

### 3) 핵심키워드 추출

In [5]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x : (x[1], x[0]), reverse=True)
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords][:n]

In [6]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names =  count_vect.get_feature_names()
tfidf_vect = tfidf_transformer.transform(count_vect.transform([doc]))
sorted_keywords = sort_keywords(tfidf_vect.tocoo())
 
# # 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)
 
print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


===== 원문 =====
과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자

=== 핵심키워드 ===
('플랫', 0.25926762688545546)
('센터', 0.22953363480835778)
('계획', 0.22222939447324755)
('활용', 0.18780024666138362)
('정통부', 0.18519116206103964)


In [7]:
tfidf_vect.tocoo().toarray()

array([[0.        , 0.02086669, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.02988221, 0.        , 0.03703823,
        0.        , 0.03703823, 0.03703823, 0.        , 0.08964664,
        0.03703823, 0.        , 0.        , 0.02988221, 0.        ,
        0.        , 0.03703823, 0.        , 0.        , 0.02480494,
        0.07407646, 0.11952885, 0.        , 0.03703823, 0.22222939,
        0.03703823, 0.04173339, 0.03703823, 0.07407646, 0.03703823,
        0.02988221, 0.03703823, 0.18519116, 0.05976443, 0.        ,
        0.        , 0.03703823, 0.        , 0.07407646, 0.        ,
        0.07407646, 0.03703823, 0.17929328, 0.03703823, 0.02988221,
        0.        , 0.03703823, 0.02988221, 0.        , 0.03703823,
        0.08964664, 0.        , 0.        , 0.02480494, 0.03703823,
        0.        , 0.05976443, 0.04960987, 0.        , 0.14941106,
        0.        , 0.        , 0.03703823, 0.05976443, 0.        ,
        0.        , 0.03703823, 0.        , 0.03


---


## 실습 2. gensim 활용


### 1) 전처리

In [8]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

### 2) TF-IDF 계산

In [9]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

document_ls = [doc.split() for doc in preprocessed_docs]
dct = Dictionary(document_ls) # 인덱스(key) - 단어(value) 인 딕셔너리 생성
corpus = [dct.doc2bow(doc) for doc in document_ls] # 각 문서에 포함된 단어를 인덱스로 변환하여 corpus 생성
tfidf = TfidfModel(corpus) # TF-IDF 산출

C:\Users\andyj\anaconda3\envs\abangues\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 3) 핵심키워드 추출

In [10]:
def sort_keywords(tfidf):
    return sorted(tfidf, key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [11]:
doc = corpus[0]

sorted_keywords = sort_keywords(tfidf[doc]) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(dct, sorted_keywords, 5)

print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


=== 핵심키워드 ===
('플랫', 0.2495222182663338)
('폼', 0.2495222182663338)
('계획', 0.21387618708542896)
('정통부', 0.17823015590452412)
('위한', 0.17823015590452412)


In [12]:
tfidf[doc]

[(0, 0.020294121162935892),
 (1, 0.020294121162935892),
 (2, 0.004942211144966953),
 (3, 0.040588242325871784),
 (4, 0.020294121162935892),
 (5, 0.03564603118090483),
 (6, 0.03564603118090483),
 (7, 0.03564603118090483),
 (8, 0.07919680641585028),
 (9, 0.06088236348880767),
 (10, 0.03564603118090483),
 (11, 0.040588242325871784),
 (12, 0.020294121162935892),
 (13, 0.03564603118090483),
 (14, 0.020294121162935892),
 (15, 0.03564603118090483),
 (16, 0.020294121162935892),
 (17, 0.011313829487978612),
 (18, 0.07129206236180965),
 (19, 0.08117648465174357),
 (20, 0.03564603118090483),
 (21, 0.21387618708542896),
 (22, 0.03564603118090483),
 (23, 0.009884422289933905),
 (24, 0.03564603118090483),
 (25, 0.07129206236180965),
 (26, 0.03564603118090483),
 (27, 0.020294121162935892),
 (28, 0.03564603118090483),
 (29, 0.17823015590452412),
 (30, 0.040588242325871784),
 (31, 0.03564603118090483),
 (32, 0.07129206236180965),
 (33, 0.07129206236180965),
 (34, 0.03564603118090483),
 (35, 0.121764726



---



# Textrank
https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

<img src="https://3.bp.blogspot.com/-yp0Lr3ec5EY/XIs6znCcO_I/AAAAAAAAAPY/xtZxe_OYtH0xeuWsp4Qd4DQrunGMpVQmQCLcBGAs/s640/keyword-extraction-textrank.png" />

## 실습 1. 행렬 활용 


In [13]:
tokens = ['딸기', '바나나', '사과', '딸기', '파인애플'] # 문서 토큰화
nodes = ['바나나', '사과', '파인애플', '딸기'] # unique한 토큰
vocab = nodes # 문서에서 분석할 타겟 토큰

# [vocab2idx[token] for token in vocab]
# vocab2idx
vocab2idx = {vocab[i]:i for i in range(0, len(vocab))} #vocab을 인덱스로 변환
idx2vocab = {i:vocab[i] for i in range(0, len(vocab))} #인덱스를 vocab으로 변환
vocab2idx

{'바나나': 0, '사과': 1, '파인애플': 2, '딸기': 3}

### 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

In [14]:
import numpy as np
import math
vocab_len = len(vocab)
# 행렬로 계산하기
# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len),dtype=np.float32)

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 2 # hyperparameter => 묶는 단어의 개수
covered_cooccurence = [] #노드 사이의 윈도우 중복으로 계산하지 않기 위해서 계산한 윈도우를 리스트에 저장 (for efficiency) 

# tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']
for window_start in range(0, (len(tokens) - window_size + 1)): # 마지막 딸기,파인애플까지 가기 위해
    window = tokens[window_start: window_start + window_size] # 첫 번째 윈도우 = tokens[0:2]
    
    for i in range(window_size):
        for j in range(i+1, window_size):
            if(window[i] in vocab, window[j] in vocab):
                index_i = i + window_start
                index_j = j + window_start
                
                if [index_i, index_j] not in covered_cooccurence:
                    # 사과랑 딸기가 연결되어있으면 딸기랑 사과도 연결
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
                    covered_cooccurence.append((index_i, index_j))

for i in range(vocab_len):
    row_sum = weighted_edge[i].sum()
    print(f"{i} : {row_sum}")
    weighted_edge[i] = weighted_edge[i]/row_sum if row_sum > 0 else 0
    # row_sum이 0인 경우에 에러 나오기 때문. 다른 단어와 아무 관계 없을 때.

print(weighted_edge)

0 : 2.0
1 : 2.0
2 : 1.0
3 : 3.0
[[0.         0.5        0.         0.5       ]
 [0.5        0.         0.         0.5       ]
 [0.         0.         0.         1.        ]
 [0.33333334 0.33333334 0.33333334 0.        ]]


### 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

In [15]:
MAX_ITERATIONS = 50
d=0.85 #damping factor
threshold = 0.0001 #convergence threshold

# vocab의 스코어를 계산하는 과정
for iter in range(MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(vocab_len):
        summation = 0
        for j in range(vocab_len):
            summation += weighted_edge[j][i] * score[j]
# Damping Factor 적용    
        score[i] = (1-d) + d*summation
    print("****score****")
    print(score)
    
    # token들의 |pre_score - score의 차의 합|이 0.0001 이하일 때
    # 0.0001이하 이면 미묘한 차이 이기 때문에 break
    if np.sum(np.fabs(prev_score - score)) <= threshold: #fabs => 절대값
        break

print("****final****")
print(score)

****score****
[0.85833335 0.798125   0.43333334 1.2223282 ]
****score****
[0.83552945 0.85142636 0.49632633 1.2888336 ]
****score****
[0.8770257 0.8879055 0.5151695 1.3379898]
****score****
[0.90645695 0.91434133 0.52909714 1.3735719 ]
****score****
[0.9277738 0.9334826 0.5391787 1.3993359]
****score****
[0.9432086  0.94734216 0.5464785  1.4179908 ]
****score****
[0.9543845 0.9573775 0.5517641 1.4314983]
****score****
[0.9624766 0.9646438 0.5555912 1.4412787]
****score****
[0.9683359 0.9699051 0.5583623 1.4483604]
****score****
[0.97257847 0.97371465 0.56036884 1.4534881 ]
****score****
[0.97565037 0.97647303 0.56182164 1.4572009 ]
****score****
[0.97787464 0.9784703  0.5628736  1.4598892 ]
****score****
[0.97948515 0.97991645 0.5636353  1.4618356 ]
****score****
[0.98065126 0.9809635  0.56418675 1.463245  ]
****score****
[0.9814956  0.98172176 0.5645861  1.4642656 ]
****score****
[0.98210704 0.9822708  0.56487525 1.4650046 ]
****score****
[0.9825497  0.9826683  0.56508464 1.4655396 ]


### 핵심 단어 추출

In [16]:
# score 리스트의 인덱스를 value가 작은 순으로 정렬
sorted_index = np.flip(np.argsort(score),0) # 행 기준으로 flip

n = 4

print("\n=== 핵심키워드 ===")
for i in range(0,n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
딸기 : 1.4668667
사과 : 0.98365426
바나나 : 0.98364776
파인애플 : 0.5656039


In [17]:
sorted_index[0]

3

## 연습

In [20]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
    headers={"user-agent":"Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, "html.parser")
    content = soup.select_one("#articleBodyContents").get_text().replace("\n", "")
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
  
    start_pos = re.search(r"\w+@\w+\.\w+(.\w+)?", content).start() #뒤에 이메일 형식
    content = content[:start_pos-1]
    return content

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc = re.sub("[^가-힣 \d]", "", doc) # 특수 문자 제거
doc

'과기정통부 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도개방성 강화이데일리 이재운 기자 국가 차원의 빅데이터 활용 시대가 열린다 새로운 산업 창출과 기존 산업의 변화에 이르는 혁신성장을 위한 센터가 문을 연다 10개 분야에 걸쳐 데이터 경제의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 빅데이터 플랫폼 및 센터 출범식 행사를 개최했다 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다10개 분야 100개 센터3년간 1516억원 투입이미지 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호보안에 대한 중요성을 강조하기도 했다이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산구축하고 플랫폼은 이를 수집분석유통하는 역할을 담당한다 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며 올해 우선 640억원 규모의 사업을 추진하고 있다대상 분야는 금융카드 환경한국수자원공사 문화한국문화정보원 교통한국교통연구원 헬스케어국립암센터 유통소비매일방송 통신 중소기업더존비즈온 지역경제경기도청 산림한국임업진흥원 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원 운영할 계획이다 이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다주요 활용 전략사례를 보면 빅데이터 활용을 통해 신 시장을 창출하는 방안을 담고 있다 금융 플랫폼의 경우 소상공인 신용평가 고도화 등을 통해 금융 취약 계층 대상 중금리 대출이

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [145]:
nodes = [t[0] for t in tokens if t[1] in ['NNG', 'NNP']]
vocab = list(set(nodes))


tokens = [ token for token in mecab.pos(doc) ]
nodes = [t[0] for t in tokens]
vocab = [t[0] for t in tokens if t[1] in ['NNG', 'NNP'] and len(t[0]) > 1]

# 확인
print(nodes[:10])
print(vocab[:10])

['과기', '정통부', '22', '일', '유영민', '장관', '등', '참석', '해', '기념행사']
['과기', '정통부', '유영민', '장관', '참석', '기념행사', '투입', '여종', '데이터', '구축']


### 2) Unique한 토큰 목록 생성

그래프 생성을 위해서 Unique한 토큰 목록 생성

In [146]:
# Set로 중복 제거 후 리스트
vocab = list(set(vocab))

vocab2idx = {vocab[i]:i for i in range(len(vocab))}
idx2vocab = {i:vocab[i] for i in range(len(vocab))}

In [147]:

import numpy as np
import math

vocab_len = len(vocab2idx)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len),dtype=np.float32)

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 3
covered_coocurrences = []

for window_start in range(len(nodes) - window_size + 1):
    window = nodes[window_start:window_start+window_size]
    for i in range(window_size):
        for j in range(i+1, window_size):
            if window[i] in vocab and window[j] in vocab:
                index_i = window_start + i
                index_j = window_start + j

                if (index_i, index_j) not in covered_coocurrences:
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
                    covered_coocurrences.append((index_i, index_j))

for i in range(vocab_len):
    row_sum = weighted_edge[i].sum()
    weighted_edge[i] = weighted_edge[i]/row_sum if row_sum > 0 else 0

MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(MAX_ITERATIONS):
    prev_score = np.copy(score)

    for i in range(vocab_len):
        summation = 0
        for j in range(vocab_len):
            if weighted_edge[j][i] != 0:
                summation += weighted_edge[j][i] * prev_score[j]

        score[i] = (1 - d) * d*summation

    if np.sum(np.fabs(prev_score -  score)) <= threshold:
        break


sorted_index = np.flip(np.argsort(score), 0)

n = 5


print("\n=== 핵심키워드 ===")
for i in range(0,n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
데이터 : 7.405189e-07
센터 : 2.49831e-07
한국 : 2.0378042e-07
활용 : 1.9834519e-07
대한 : 1.9389464e-07


---

## 실습 2. 그래프 활용 

### 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 없음을 의미
*   모든 노드는 1로 초기화

In [18]:
import numpy as np
import math
import networkx as nx

# 윈도내 동시 등장한 토큰으로 그래프를 생성
def connect(vocab, nodes) :            
    window_size = 3
    
    edges = []
    for window_start in range(len(nodes)-window_size+1):
        window = nodes[window_start: window_start + window_size]
        for i in range(window_size):
            for j in range(i+1, window_size):
                if window[i] in vocab and window[j] in vocab:
                    edges.append((window[i], window[j]))
        
    return edges

graph=nx.diamond_graph()
graph.clear() 
graph.add_nodes_from(vocab) #node 등록
graph.add_edges_from(connect(vocab,nodes)) #edge 연결    

### 스코어 계산 및 핵심키워드 추출

In [151]:
scores = nx.pagerank(graph) #pagerank 계산

rank = sorted(scores.items(), key=lambda x: x[1], reverse=True) #score 역순 정렬
print("\n=== 핵심키워드 ===")
rank[:5]


=== 핵심키워드 ===


[('데이터', 0.04661070207125959),
 ('센터', 0.01676522225593208),
 ('대한', 0.012413811834749081),
 ('활용', 0.012361319397854264),
 ('한국', 0.012348645054362992)]

---



---



## gensim 활용

In [162]:
from gensim.summarization import keywords
keywords(" ".join(vocab), words=5).split('\n')

['얼라이언스', '수자원', '청사진 대통령', '경쟁력']